<a href="https://colab.research.google.com/github/sarangbishal/Ballot-Paper-Counting-System/blob/master/Balloot_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import all required libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt

import imutils
import cv2

from collections import Counter
from imutils.perspective import four_point_transform
from imutils import contours

### Read Image and detect edges using cv2 Canny algorithm

In [0]:
FILE_NAME = "Kathmandu10.jpg"

In [0]:
image = cv2.imread(FILE_NAME)
image_matplotlib_compat = imutils.opencv2matplotlib(image)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# edged = cv2.Canny(blurred, 75, 200)
# auto_edged = imutils.auto_canny(gray)

# plt.imsave("image_1.jpg", edged)
# plt.imshow(auto_edged)

### Add contour detection and isolate candidate boxes

In [99]:
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
rows = []

most_common_area = Counter([cv2.contourArea(c) for c in cnts if len(cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)) == 4]).most_common(1)[0][0]
area_delta  = 999
i = 0
# print(most_common_area)
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    current_contour_area = cv2.contourArea(c)
    # print(current_contour_area)
    if len(approx) == 4  and abs(current_contour_area - most_common_area) <= area_delta:
        rows.append(c)
    
print(len(rows))

39


### Arrange each boxes in the order as they appear on paper

In [100]:
rows = contours.sort_contours(rows, method="top-to-bottom")[0]

k = 0
sorted_boxes = []

for (q, i) in enumerate(np.arange(0, len(rows), 3)):
    cnts = contours.sort_contours(rows[i:i + 3], method = "left-to-right")[0]
    sorted_boxes.extend(cnts)

print(len(sorted_boxes))

for c in sorted_boxes:
    k += 1
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    paper = four_point_transform(image_matplotlib_compat, approx.reshape(4, 2))
    plt.imsave("naya/test_" + str(k) + ".jpg", paper)

    
plt.imsave("ou.jpg", image_matplotlib_compat)

39
